In [75]:
import pandas as pd
import os
import json
import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import csv
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [3]:
doi_index = []
text = []

In [4]:
pathacs = '/Users/juliachotoo/ScrapyArticles/scrapedjson/acs2/'

for filename in os.listdir(pathacs):
    with open(pathacs + filename, 'r'):
        resultacs = json.load(open(pathacs + filename, mode='r'))
        doi_index.append(resultacs['doi'])
        text.append(resultacs['text'])

In [5]:
pathspr = '/Users/juliachotoo/ScrapyArticles/scrapedjson/spr2/'

for filename in os.listdir(pathspr):
    with open(pathspr + filename, 'r'):
        resultspr = json.load(open(pathspr + filename, mode='r'))
        doi_index.append(resultspr['doi'])
        text.append(resultspr['text'])

In [28]:
pathlabels = '/Users/juliachotoo/ScrapyArticles/datalabels.csv'
labels = []

with open(pathlabels, 'r') as csvfile:
    resultlabels = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in resultlabels:
        labels.append(', '.join(row))

labels[0] = labels[0].replace('\ufeff', '')
print(labels)    


['0', '1', '0', '1', '0', '0', '2', '0', '2', '0', '0', '0', '0', '0', '0', '1', '0', '0', '2', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '2', '2', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '2', '1', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '2', '0', '1', '0', '0', '0', '2', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '2', '0', '1', '1', '2', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0',

In [66]:
labels.count('0')

202

In [41]:
df = pd.DataFrame({'data': text, 'labels': labels})
print(df.loc[0:9])

                                                data labels
0  10.1021/ic501413r Metal–organic frameworks (MO...      0
1  10.1021/jp102881e  Top of PageAbstractIntroduc...      1
2  10.1021/acs.analchem.5b00391 Metal–organic fra...      0
3  10.1021/ie071645b  Top of PageAbstract1. Intro...      1
4  10.1021/ic502478u Metal–organic frameworks (MO...      0
5  10.1021/cm4034319 Top of PageAbstractIntroduct...      0
6  10.1021/acs.langmuir.5b04185 In the original a...      2
7  10.1021/ie500310c Top of PageAbstractIntroduct...      0
8  10.1021/ie051056a  Top of PageAbstractIntroduc...      2
9  10.1021/ja5022014 Top of PageAbstractIntroduct...      0


In [57]:
X = df.data
y = df[df.columns[1]]
print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(267,)
(267,)
(200,)
(67,)
(200,)
(67,)


In [58]:
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.fit_transform(X_train)


X_test_dtm = vect.transform(X_test)
X_test_dtm

<67x32574 sparse matrix of type '<class 'numpy.int64'>'
	with 65238 stored elements in Compressed Sparse Row format>

In [69]:
tf_transformer = TfidfTransformer(use_idf=False)
X_train_tf = tf_transformer.fit_transform(X_train_dtm)
X_test_tf = tf_transformer.transform(X_test_dtm)

In [70]:
nb = MultinomialNB()

In [84]:
%time nb.fit(X_train_dtm, y_train)

CPU times: user 6.91 ms, sys: 2.16 ms, total: 9.07 ms
Wall time: 7.07 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [85]:
y_pred_class = nb.predict(X_test_dtm)

In [86]:
metrics.accuracy_score(y_test, y_pred_class)

0.79104477611940294

In [87]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[49,  3,  0],
       [ 3,  4,  0],
       [ 5,  3,  0]])

In [89]:
X_test[(y_pred_class==1) & (y_test==0)]

/Users/juliachotoo/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


Series([], Name: data, dtype: object)

In [76]:
logreg = LogisticRegression()

In [77]:
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 784 ms, sys: 21.1 ms, total: 805 ms
Wall time: 595 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [78]:
y_pred_class2 = logreg.predict(X_test_dtm)

In [80]:
metrics.accuracy_score(y_test, y_pred_class2)

0.77611940298507465

In [83]:
metrics.confusion_matrix(y_test, y_pred_class2)

array([[48,  4,  0],
       [ 3,  4,  0],
       [ 7,  1,  0]])